***
# **FloodProofs Labs - HMC Runner** 

<img style="float: left; padding-right: 80px; padding-left: 5px;" src="img/logo_hmc.png" width="200px" align=”left” >

In the laboratory of **HMC runner** we will perform the following points:
   * Configure the libraries and the dependecies of the laboratory;
   * Read the configuration file of the laboratory;
   * Read the static datasets of terrain, river networks and outlet sections;
   * Read the dynamic datasets of the time-series (collections and hydrographs);
   * Plot the position of the analyzed outlet section;
   * Plot the time-series of discharge and the time-series of the hmc average forcings.    


## **Import libraries and dependencies**

In [16]:
%matplotlib inline
%matplotlib widget

In [55]:
# Add hmc library to the paths environment
import sys
import os

folder_home_env = os.path.expanduser('~')
folder_library_hmc = "/fp_labs_hmc/library/jupyter_hmc/"

folder_library_hmc = folder_home_env + folder_library_hmc
sys.path.append(folder_library_hmc)

print(' ==> Library HMC ' + folder_library_hmc + ' added to the library environments')

 ==> Library HMC /home/fabio/fp_labs_hmc/library/jupyter_hmc/ added to the library environments


In [56]:
# Libraries
import argparse
import time
import pandas as pd

# Import method to adapt settings file
from library.jupyter_generic.lib_jupyter_utils_io import update_json_file

# Import coupler and driver classes
from hmc.driver.configuration.drv_configuration_hmc_logging import ModelLogging
from hmc.coupler.cpl_hmc_manager import ModelInitializer, ModelCleaner
from hmc.coupler.cpl_hmc_builder import ModelBuilder
from hmc.coupler.cpl_hmc_runner import ModelRunner
from hmc.coupler.cpl_hmc_finalizer import ModelFinalizer

# Info
print(' ==> Libraries loaded')

 ==> Libraries loaded


### **Configure the HMC runner laboratory**
- Set the notebook arguments:

In [57]:
# Define the algorithm and datasets configuration file(s)
script_settings_algorithm_default="fp_labs_runner_hmc_algorithm_default.json"
script_settings_algorithm_defined="fp_labs_runner_hmc_algorithm_defined.json"
script_settings_datasets_default="fp_labs_runner_hmc_datasets_default.json"
script_settings_datasets_defined="fp_labs_runner_hmc_datasets_defined.json"
# Define the time 
script_time='2021-01-21 06:00'

- Update the configuration file(s) according with the environment variable(s)

In [58]:
# Define the script time as a timestamp object
script_time = pd.Timestamp(script_time)

# Set environment variables to json settings file(s)
update_json_file(script_settings_algorithm_default, script_settings_algorithm_defined,
                 update_dict={'$ENV_HOME': folder_home_env, '$ENV_TIME': script_time.strftime("%Y%m%d_%H")})
update_json_file(script_settings_datasets_default, script_settings_datasets_defined,
                 update_dict={'$ENV_HOME': folder_home_env, '$ENV_TIME': script_time.strftime("%Y%m%d_%H")})

### **Initialize the instance of HMC model**
- Set the logging file:

In [59]:
# Set logging file
driver_hmc_logging = ModelLogging(script_settings_algorithm_defined)
log_stream = driver_hmc_logging.configure_logging()

# Set the verbose debug level
log_stream.setLevel(logging.WARNING)


- Configure the initializer model class

In [60]:
# Set the verbose debug level
log_stream.setLevel(logging.ERROR)

# Configure model initializer class
driver_hmc_initializer = ModelInitializer(file_algorithm=script_settings_algorithm_defined,
                                          file_datasets=script_settings_datasets_defined, time=script_time)

- Configure the algorithm and the ancillary datasets

In [61]:
# Configure algorithm
time_series_collections, time_info_collections, \
    run_info_collections, run_cline_collections = driver_hmc_initializer.configure_algorithm()

In [62]:
# Configure ancillary datasets
ancillary_datasets_collections = driver_hmc_initializer.configure_ancillary_datasets(time_info_collections)

### **Build the instance of HMC model**
- Configure the builder model class

In [63]:
# Set the verbose debug level
log_stream.setLevel(logging.ERROR)

# Configure model builder class
driver_hmc_builder = ModelBuilder(
    obj_geo_reference=driver_hmc_initializer.dset_ref_geo,
    obj_args=driver_hmc_initializer.obj_args,
    obj_run=driver_hmc_initializer.obj_run,
    obj_ancillary=driver_hmc_initializer.obj_ancillary)

- Configure the static and the dynamic datasets

In [64]:
# Configure static datasets
static_datasets_collections = driver_hmc_builder.configure_static_datasets(ancillary_datasets_collections)

2021-01-25 15:49:17,372 hmc_logger   ERROR     ===> File static /home/fabio/fp_labs_datasets/data_static/shapefile/fp_sections_marche.shp is mandatory! Execution exit. drv_dataset_hmc_io_type.py:[200    - read_filename_static()] 
2021-01-25 15:49:17,372 hmc_logger   ERROR     ===> File static /home/fabio/fp_labs_datasets/data_static/shapefile/fp_sections_marche.shp is mandatory! Execution exit. drv_dataset_hmc_io_type.py:[200    - read_filename_static()] 
2021-01-25 15:49:17,372 hmc_logger   ERROR     ===> File static /home/fabio/fp_labs_datasets/data_static/shapefile/fp_sections_marche.shp is mandatory! Execution exit. drv_dataset_hmc_io_type.py:[200    - read_filename_static()] 
2021-01-25 15:49:17,372 hmc_logger   ERROR     ===> File static /home/fabio/fp_labs_datasets/data_static/shapefile/fp_sections_marche.shp is mandatory! Execution exit. drv_dataset_hmc_io_type.py:[200    - read_filename_static()] 
2021-01-25 15:49:17,372 hmc_logger   ERROR     ===> File static /home/fabio/fp_l

OSError: File not found

In [51]:
# Configure dynamic datasets
forcing_datasets_collections = driver_hmc_builder.configure_dynamic_datasets(
    time_series_collections, time_info_collections, static_datasets_collections, ancillary_datasets_collections)

### **Run the instance of HMC model**
- Configure the runner model class

In [30]:
# Set the verbose debug level
log_stream.setLevel(logging.ERROR)

# Configure model runner class
driver_hmc_runner = ModelRunner(time_info=time_info_collections, run_info=run_info_collections,
                                command_line_info=run_cline_collections,
                                obj_args=driver_hmc_initializer.obj_args,
                                obj_ancillary=driver_hmc_initializer.obj_ancillary)

- Execute the instance of the model

In [ ]:
# Configure the model execution
driver_hmc_runner.configure_execution(ancillary_datasets_collections)

### **Finalize the instance of the model**
- Configure the finalizer class of the model

In [ ]:
# Configure model finalizer class
driver_hmc_finalizer = ModelFinalizer(
    collection_dynamic=forcing_datasets_collections,
    obj_geo_reference=driver_hmc_initializer.dset_ref_geo,
    obj_args=driver_hmc_initializer.obj_args,
    obj_run=driver_hmc_initializer.obj_run,
    obj_ancillary=driver_hmc_initializer.obj_ancillary)

- Dump the model results in gridded, point and time-series format

In [ ]:
# Configure the outcome datasets
outcome_datasets_collections = driver_hmc_finalizer.configure_dynamic_datasets(
    time_series_collections, time_info_collections, static_datasets_collections, ancillary_datasets_collections)

- Dump the model results in collections format

In [ ]:
# Configure the summary datasets
driver_hmc_finalizer.configure_summary_datasets(
    time_series_collections, time_info_collections, static_datasets_collections, outcome_datasets_collections)